In [1]:
"""
 - Fusion des fichiers des capacités et des indisponibilités de production générés par les codes précédents (rub.3 et 4)
 - Création de colonnes calculées des puissances disponibles et de capacités maximales par filière
Created on 2024-06-15

@author: Thierry ALLEM
"""

'\n - Fusion des fichiers des capacités et des indisponibilités de production générés par les codes précédents (rub.3 et 4)\n - Création de colonnes calculées des puissances disponibles et de capacités maximales par filière\nCreated on 2024-06-15\n\n@author: Thierry ALLEM\n'

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import holidays

In [3]:
import warnings
# Pour ignorer tous les warnings
warnings.filterwarnings('ignore')

In [4]:
# ********************************************  FUSION  DES DONNEES ECO2MIX - CAPACITES MAXIMALES DE PRODUCTION ET INDISPONIBILITES DE PRODUCTION *********************************************************************

# Importation du fichier eco2mix_regional_def préparé, incluant les TCO
df_eco2_cap = pd.read_csv('df_eco2_cleaned_tco_tch.csv',sep = ';', encoding='latin-1')

df_eco2_cap.info()

df_eco2_cap_copy = df_eco2_cap.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121396 entries, 0 to 2121395
Data columns (total 40 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0               int64  
 1   region                   object 
 2   date                     object 
 3   heure                    object 
 4   consommation             float64
 5   mois                     int64  
 6   annee                    int64  
 7   date_ym                  object 
 8   date_heure               object 
 9   thermique                float64
 10  nucleaire                float64
 11  eolien                   float64
 12  solaire                  float64
 13  hydraulique              float64
 14  pompage                  float64
 15  bioenergies              float64
 16  ech_physiques            float64
 17  stockage_batterie        float64
 18  destockage_batterie      float64
 19  tch_thermique_rte        float64
 20  tch_nucleaire_rte        float64
 21  tch_eoli

In [5]:
df_eco2_cap.head()

,Unnamed: 0,region,date,heure,consommation,mois,annee,date_ym,date_heure,thermique,...,tco_solaire,tco_hydraulique,tco_pompage,tco_bioenergies,tco_export,tco_import,tco_stockage_batterie,tco_destockage_batterie,total_tco,tco_prod_tot
0,12,CENTRE VAL DE LOIRE,2013-01-01,00:30:00,2476.0,1,2013,2013-01,2013-01-01 00:30:00,90.0,...,0.0,1.37,0.00,1.01,-293.50,0.00,0.0,0.0,99.96,393.46
1,13,PAYS DE LA LOIRE,2013-01-01,00:30:00,3595.0,1,2013,2013-01,2013-01-01 00:30:00,127.0,...,0.0,0.00,0.00,0.89,0.00,90.46,0.0,0.0,99.94,9.49
2,14,GRAND EST,2013-01-01,00:30:00,4943.0,1,2013,2013-01,2013-01-01 00:30:00,319.0,...,0.0,28.69,0.00,1.15,-143.68,0.00,0.0,0.0,99.90,243.58
3,15,ILE DE FRANCE,2013-01-01,00:30:00,9134.0,1,2013,2013-01,2013-01-01 00:30:00,685.0,...,0.0,0.00,0.00,1.55,0.00,90.75,0.0,0.0,99.98,9.23
4,16,OCCITANIE,2013-01-01,00:30:00,5228.0,1,2013,2013-01,2013-01-01 00:30:00,78.0,...,0.0,18.04,-0.25,1.55,0.00,24.35,0.0,0.0,99.96,75.86


In [6]:
# Extraction des capacités maximales de production par région, filière et date effectuée dans code distinct 'capacites_productions' (copie du code c-dessous)

In [7]:
# Importation du fichier après extraction et calcul depuis le Registre des Installations

df_result_cap_prod = pd.read_csv('df_result_cap_prod_2013_2022.csv', sep=';')

In [8]:
df_result_cap_prod.head()

,Unnamed: 0,region,filiere_production,date,puis_max_installee
0,0,AUVERGNE RHONE ALPES,bioenergies,2013-01-01 00:00:00,104.762000
1,1,AUVERGNE RHONE ALPES,eolien,2013-01-01 00:00:00,335.379400
2,2,AUVERGNE RHONE ALPES,hydraulique,2013-01-01 00:00:00,10679.463890
3,3,AUVERGNE RHONE ALPES,nucleaire,2013-01-01 00:00:00,13570.000000
4,4,AUVERGNE RHONE ALPES,solaire,2013-01-01 00:00:00,703.899945


In [9]:
# Conversion de la colonne 'date' de df_result_cap_prod en datetime
df_result_cap_prod['date'] = pd.to_datetime(df_result_cap_prod['date'])
df_eco2_cap['date_heure'] = pd.to_datetime(df_eco2_cap['date_heure'])

# Pivot afin d'obtenir une colonne pour chaque filière de production
pivot_df = df_result_cap_prod.pivot_table(index=['region', 'date'],
                                          columns='filiere_production',
                                          values='puis_max_installee').reset_index()

# Renommage des colonnes avant intégration dans df_eco2_cap
column_mapping = {
    'bioenergies': 'cap_prod_max_bioenergies',
    'eolien': 'cap_prod_max_eolien',
    'hydraulique': 'cap_prod_max_hydraulique',
    'nucleaire': 'cap_prod_max_nucleaire',
    'solaire': 'cap_prod_max_solaire',
    'thermique': 'cap_prod_max_thermique'
}

# Ajout de 'region' et 'date' au mapping si elles sont présentes
new_columns = ['region', 'date'] + [column_mapping.get(col, col) for col in pivot_df.columns if col not in ['region', 'date']]
pivot_df.columns = new_columns

# Jointure de pivot_df avec df_eco2_cap_ind sur les colonnes 'region' et 'date_heure'
df_eco2_cap = df_eco2_cap.merge(pivot_df, left_on=['region', 'date_heure'], right_on=['region', 'date'], how='left')

# Vérification que la colonne 'date' existe avant de la supprimer
if 'date' in df_eco2_cap.columns:
    df_eco2_cap.drop(columns=['date'], inplace=True)


# Vérification du résultat
df_eco2_cap.head()

,Unnamed: 0,region,date_x,heure,consommation,mois,annee,date_ym,date_heure,thermique,...,tco_prod_tot,date_y,cap_prod_max_bioenergies,cap_prod_max_eolien,geothermie,cap_prod_max_hydraulique,cap_prod_max_nucleaire,cap_prod_max_solaire,stockage_batterie_y,cap_prod_max_thermique
0,12,CENTRE VAL DE LOIRE,2013-01-01,00:30:00,2476.0,1,2013,2013-01,2013-01-01 00:30:00,90.0,...,393.46,2013-01-01 00:30:00,41.2760,661.9340,NaN,90.6510,11630.0,155.337562,NaN,156.688
1,13,PAYS DE LA LOIRE,2013-01-01,00:30:00,3595.0,1,2013,2013-01,2013-01-01 00:30:00,127.0,...,9.49,2013-01-01 00:30:00,56.5030,453.8490,NaN,4.9120,NaN,384.641840,NaN,1837.276
2,14,GRAND EST,2013-01-01,00:30:00,4943.0,1,2013,2013-01,2013-01-01 00:30:00,319.0,...,243.58,2013-01-01 00:30:00,75.9025,1802.5850,NaN,2267.9864,10820.0,495.656904,NaN,3378.019
3,15,ILE DE FRANCE,2013-01-01,00:30:00,9134.0,1,2013,2013-01,2013-01-01 00:30:00,685.0,...,9.23,2013-01-01 00:30:00,262.3540,36.9320,NaN,19.2130,NaN,107.650310,NaN,2121.295
4,16,OCCITANIE,2013-01-01,00:30:00,5228.0,1,2013,2013-01,2013-01-01 00:30:00,78.0,...,75.86,2013-01-01 00:30:00,126.3370,849.3247,NaN,4923.1960,2620.0,1039.602893,NaN,245.135


In [10]:
# **************************************************** AJOUT DES RESULTATS DES EXTRACTIONS DES INDISPONIBILITES DES MOYENS DE PRODUCTIONS *****************
# Extraction des indisponibilités des moyens de production par région, filière et date effectuée dans code distinct 'indisponibilites' (copie du code c-dessous)
# Importation du fichier après extraction et calcul depuis le Registre des Installations

In [11]:
# Importation du fichier des capacités de productions totales
df_result_ind_prod = pd.read_csv('df_result_ind_prod_prefusion.csv', sep=';')

# Conversion de la colonne 'date' de df_result_cap_prod en datetime
df_result_ind_prod['date'] = pd.to_datetime(df_result_ind_prod['date'])

df_result_ind_prod.head()

,region,filiere,date,capacite_indisponible
0,AUVERGNE RHONE ALPES,hydraulique,2013-01-01 17:30:00,180.0
1,AUVERGNE RHONE ALPES,hydraulique,2013-01-01 18:00:00,180.0
2,AUVERGNE RHONE ALPES,hydraulique,2013-01-01 18:30:00,180.0
3,AUVERGNE RHONE ALPES,hydraulique,2013-01-01 19:00:00,180.0
4,AUVERGNE RHONE ALPES,hydraulique,2013-01-04 00:30:00,259.0


In [12]:
df_result_ind_prod['filiere'].unique()

array(['hydraulique', 'nucleaire', 'thermique', 'eolien', 'bioenergies'],
      dtype=object)

In [13]:
# Pivot afin d'obtenir une colonne pour chaque filière de production
pivot_df2 = df_result_ind_prod.pivot_table(index=['region', 'date'],
                                          columns='filiere',
                                          values='capacite_indisponible').reset_index()

# Renommage des colonnes avant intégration dans df_eco2_cap_ind
column_mapping2 = {'bioenergies': 'ind_prod_bioenergies','hydraulique': 'ind_prod_hydraulique',
    'nucleaire': 'ind_prod_nucleaire','thermique': 'ind_prod_thermique'}

# Ajout de 'region' et 'date' au mapping si elles sont présentes
new_columns2 = ['region', 'date'] + [column_mapping2.get(col, col) for col in pivot_df2.columns if col not in ['region', 'date']]
pivot_df2.columns = new_columns2

# Jointure de pivot_df2 avec df_eco2_cap_ind sur les colonnes 'region' et 'date_heure'
df_eco2_cap_ind = df_eco2_cap.merge(pivot_df2, left_on=['region', 'date_heure'], right_on=['region', 'date'], how='left')

# Vérification que la colonne 'date' existe avant de la supprimer
if 'date' in df_eco2_cap_ind.columns:
    df_eco2_cap_ind.drop(columns=['date'], inplace=True)


# Remplacement des valeurs manquantes par la valeur 0
df_eco2_cap_ind.fillna(0, inplace = True)
print(df_eco2_cap_ind.isna().sum())

# Arrondi des valeurs des capacités et indisponibilités à 2 décimales
df_eco2_cap_ind.iloc[:, 9:19] = df_eco2_cap_ind.iloc[:, 9:19].round(2)

# AJOUT DE COLONNES DE DONNEES TEMPORELLES

# Ajout de la colonne 'annee'
df_eco2_cap_ind['annee'] = df_eco2_cap_ind['date_heure'].dt.year

# Ajout de la colonne 'mois_nom'
df_eco2_cap_ind['mois_nom'] = df_eco2_cap_ind['date_heure'].dt.strftime('%b')

# Ajout de la colonne 'mois'
df_eco2_cap_ind['mois'] = df_eco2_cap_ind['date_heure'].dt.month

# Ajout de la colonne 'jour_nom'
df_eco2_cap_ind['jour_nom'] = df_eco2_cap_ind['date_heure'].dt.strftime('%a')

# Ajout de la colonne 'jour'
df_eco2_cap_ind['jour'] = df_eco2_cap_ind['date_heure'].dt.day

# Ajout de la colonne 'jour_numero'
df_eco2_cap_ind['jour_numero'] = df_eco2_cap_ind['date_heure'].dt.strftime('%j')

# Ajout de la colonne 'semaine'
df_eco2_cap_ind['semaine'] = df_eco2_cap_ind['date_heure'].dt.strftime('%U')

# Ajout de la colonne 'heure'
# df_eco2_cap_ind['heure'] = df_eco2_cap_ind['date_heure'].dt.strftime('%X')

# Ajout de la colonne 'jour_ferie'
jours_feries_fr = holidays.France(years=range(2013, 2023))
df_eco2_cap_ind['jour_ferie'] = df_eco2_cap_ind['date_heure'].dt.strftime('%Y-%m-%d').isin(jours_feries_fr)

Unnamed: 0                  0
region                      0
date_x                      0
heure                       0
consommation                0
mois                        0
annee                       0
date_ym                     0
date_heure                  0
thermique                   0
nucleaire                   0
eolien_x                    0
solaire                     0
hydraulique                 0
pompage                     0
bioenergies                 0
ech_physiques               0
stockage_batterie_x         0
destockage_batterie         0
tch_thermique_rte           0
tch_nucleaire_rte           0
tch_eolien_rte              0
tch_solaire_rte             0
tch_hydraulique_rte         0
tch_bioenergies_rte         0
export                      0
import                      0
tco_thermique               0
tco_nucleaire               0
tco_eolien                  0
tco_solaire                 0
tco_hydraulique             0
tco_pompage                 0
tco_bioene

In [14]:
#  ************ AJOUT D'UNE COLONNE PORTANT LES NOMS DE REGIONS ABREGES SELON LA NORME ISO 3166-2 ****************************************************
        # Insertion d'une nouvelle colonne 'region_abr' à l'index 1
df_eco2_cap_ind.insert(1, 'region_abr', df_eco2_cap_ind['region'])

        # Affectation des valeurs de 'region_ncc' à 'region_abr'
df_eco2_cap_ind['region_abr'] = df_eco2_cap_ind['region']

        # Renommage des valeurs de 'region_abr'
dictionnaire_region_abr= {"PROVENCE ALPES COTE D AZUR" : "FR-PAC",
                      "BOURGOGNE FRANCHE COMTE" : "FR-BFC",
                      "BRETAGNE" : "FR-BRE",
                      "NORMANDIE" : "FR-NOR",
                      "NOUVELLE AQUITAINE" : "FR-NAQ",
                      "HAUTS DE FRANCE" : "FR-HDF",
                      "ILE DE FRANCE" : "FR-IDF",
                      "AUVERGNE RHONE ALPES":"FR-ARA",
                      "GRAND EST":"FR-GES",
                      "CENTRE VAL DE LOIRE":"FR-CVL",
                      "OCCITANIE":"FR-OCC",
                      "PAYS DE LA LOIRE":"FR-PDL"}
df_eco2_cap_ind['region_abr']=df_eco2_cap_ind['region_abr'].replace(dictionnaire_region_abr)
print("Noms abrégés des régions : ", df_eco2_cap_ind['region_abr'].unique())

Noms abrégés des régions :  ['FR-CVL' 'FR-PDL' 'FR-GES' 'FR-IDF' 'FR-OCC' 'FR-NOR' 'FR-PAC' 'FR-BFC'
 'FR-HDF' 'FR-BRE' 'FR-ARA' 'FR-NAQ']


In [15]:
# Renommage de colonnes
df_eco2_cap_ind = df_eco2_cap_ind.rename(columns={'eolien_x':'eolien', 'solaire_x' : 'solaire'})

In [16]:
print("Noms abrégés des régions après renommage: ", df_eco2_cap_ind['region_abr'].unique())

Noms abrégés des régions après renommage:  ['FR-CVL' 'FR-PDL' 'FR-GES' 'FR-IDF' 'FR-OCC' 'FR-NOR' 'FR-PAC' 'FR-BFC'
 'FR-HDF' 'FR-BRE' 'FR-ARA' 'FR-NAQ']


In [17]:
# **************************************************** AJOUT DES CALCULS  DES PUISSANCES DISPONIBLES PAR FILIERE *****************
# Ajout des capacités de production disponibles en tenant compte des indisponibilités des moyens de production


thermique_p_disp = df_eco2_cap_ind['cap_prod_max_thermique'] - df_eco2_cap_ind['ind_prod_thermique']
nucleaire_p_disp = df_eco2_cap_ind['cap_prod_max_nucleaire'] - df_eco2_cap_ind['ind_prod_nucleaire']
hydraulique_p_disp = df_eco2_cap_ind['cap_prod_max_hydraulique'] - df_eco2_cap_ind['ind_prod_hydraulique']
bioenergies_p_disp = df_eco2_cap_ind['cap_prod_max_bioenergies'] - df_eco2_cap_ind['ind_prod_bioenergies']
df_eco2_cap_ind['thermique_p_disp'] = np.around(thermique_p_disp,0)
df_eco2_cap_ind['nucleaire_p_disp'] = np.around(nucleaire_p_disp,0)
df_eco2_cap_ind['hydraulique_p_disp'] = np.around(hydraulique_p_disp,0)
df_eco2_cap_ind['bioenergies_p_disp'] = np.around(bioenergies_p_disp,0)
    
#   A noter que les registres des indisponibilités des filières 'eolien' et solaire ne sont pas disponibles auprès de RTE
#       Renommage des colonnes cap_prod_max_eolien et cap_prod_max_eolien (à défaut de données, on suppose les indisponibilités nulles)

dictionnaire_cap_prod_disp= {'cap_prod_max_eolien':'eolien_p_disp', 'cap_prod_max_solaire':'solaire_p_disp'} 
df_eco2_cap_ind=df_eco2_cap_ind.rename(dictionnaire_cap_prod_disp, axis = 1)

In [18]:
# CALCUL DES TCH (Taux de Charge des filières)  PAR REGION ET PAR FILIERE - Les batteries de stockage et les échanges physiques (import/export)
#ne sont pas inclus dans les TCH

tch_thermique = np.where(df_eco2_cap_ind['thermique_p_disp'] > 0, df_eco2_cap_ind['thermique']/df_eco2_cap_ind['thermique_p_disp']*100, 0)
tch_nucleaire= np.where(df_eco2_cap_ind['nucleaire_p_disp'] > 0, df_eco2_cap_ind['nucleaire']/df_eco2_cap_ind['nucleaire_p_disp']*100, 0)
tch_eolien = np.where(df_eco2_cap_ind['eolien_p_disp'] > 0, df_eco2_cap_ind['eolien']/df_eco2_cap_ind['eolien_p_disp']*100, 0)
tch_solaire= np.where(df_eco2_cap_ind['solaire_p_disp'] > 0, df_eco2_cap_ind['solaire']/df_eco2_cap_ind['solaire_p_disp']*100, 0)

tch_hydraulique=np.where(df_eco2_cap_ind['hydraulique_p_disp']>0, df_eco2_cap_ind['hydraulique']/df_eco2_cap_ind['hydraulique_p_disp']*100, 0)
tch_bioenergies=np.where(df_eco2_cap_ind['bioenergies_p_disp']>0, df_eco2_cap_ind['bioenergies']/df_eco2_cap_ind['bioenergies_p_disp']*100, 0)

# Calcul des TCH totaux par région (regroupement de toutes les filières de production)
region_p_disp = (df_eco2_cap_ind['thermique_p_disp'] + df_eco2_cap_ind['nucleaire_p_disp'] + df_eco2_cap_ind['eolien_p_disp']+ 
                 df_eco2_cap_ind['solaire_p_disp'] + df_eco2_cap_ind['hydraulique_p_disp']+df_eco2_cap_ind['bioenergies_p_disp'])
df_eco2_cap_ind['region_p_disp'] = round(region_p_disp, 0)

region_p_reelle = (df_eco2_cap_ind['thermique'] + df_eco2_cap_ind['nucleaire'] + df_eco2_cap_ind['eolien'] + df_eco2_cap_ind['solaire']
                       + df_eco2_cap_ind['hydraulique'] + df_eco2_cap_ind['bioenergies'])
df_eco2_cap_ind['region_p_reelle'] = round(region_p_reelle, 0) 

tch_region = np.where(df_eco2_cap_ind['region_p_disp'] > 0, df_eco2_cap_ind['region_p_reelle'] / df_eco2_cap_ind['region_p_disp']*100, 0)

df_eco2_cap_ind['tch_thermique'] = np.around(tch_thermique,2)
df_eco2_cap_ind['tch_nucleaire'] = np.around(tch_nucleaire,2)
df_eco2_cap_ind['tch_eolien'] = np.around(tch_eolien,2)
df_eco2_cap_ind['tch_solaire'] = np.around(tch_solaire,2)
df_eco2_cap_ind['tch_hydraulique'] = np.around(tch_hydraulique,2)
df_eco2_cap_ind['tch_bioenergies'] = np.around(tch_bioenergies,2)
df_eco2_cap_ind['tch_region'] = np.around(tch_region,2)

In [19]:
# Suppression de la colonne 'Unnamed: 0'
# col_to_drop = ['Unnamed: 0','nature','date_heure_utc','date_y']
col_to_drop = ['Unnamed: 0','date_y']
df_eco2_cap_ind = df_eco2_cap_ind.drop(col_to_drop, axis=1)

# Renommage de colonnes
df_eco2_cap_ind = df_eco2_cap_ind.rename(columns={'date_x':'date'})
df_eco2_cap_ind = df_eco2_cap_ind.rename(columns={'stockage_batterie_x':'stockage_batterie'})
df_eco2_cap_ind = df_eco2_cap_ind.rename(columns={'date_ym':'annee_mois'})

In [20]:
# ********** AJ0UT DE LA CAPACITE MAXIMALE DE PRODUCTION DE CHAQUE REGION SANS PRISE EN COMPTE DES INDISPONIBILITES *****************

region_p_max = (df_eco2_cap_ind['cap_prod_max_thermique'] + df_eco2_cap_ind['cap_prod_max_nucleaire'] + df_eco2_cap_ind['eolien_p_disp']+ 
                 df_eco2_cap_ind['solaire_p_disp'] + df_eco2_cap_ind['cap_prod_max_hydraulique']+df_eco2_cap_ind['cap_prod_max_bioenergies'])
df_eco2_cap_ind['region_p_max'] = np.around(region_p_max,2)


In [21]:
# Réorganisation de l'ordre des colonnes

ordre_desire2 = ['region_abr', 'region', 'date_heure', 'date', 'heure', 'annee_mois', 'annee', 'mois', 'mois_nom', 
                 'jour', 'jour_nom', 'jour_numero','semaine', 'jour_ferie',
                 'consommation', 'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies','pompage', 'ech_physiques','export', 'import',
                 'stockage_batterie', 'destockage_batterie',
                 'tch_thermique_rte', 'tch_nucleaire_rte','tch_eolien_rte','tch_solaire_rte','tch_hydraulique_rte','tch_bioenergies_rte',
                 'cap_prod_max_thermique', 'ind_prod_thermique', 'thermique_p_disp', 'cap_prod_max_nucleaire', 'ind_prod_nucleaire', 'nucleaire_p_disp',
                 'eolien_p_disp','solaire_p_disp', 'cap_prod_max_hydraulique', 'ind_prod_hydraulique', 'hydraulique_p_disp', 
                 'cap_prod_max_bioenergies','ind_prod_bioenergies','bioenergies_p_disp', 'region_p_max',
                 'tco_thermique', 'tco_nucleaire', 'tco_eolien','tco_solaire', 'tco_hydraulique', 'tco_pompage', 'tco_bioenergies',
                 'tco_export','tco_import', 'tco_stockage_batterie', 'tco_destockage_batterie']



df_eco2_cap_ind = df_eco2_cap_ind[ordre_desire2]


df_eco2_cap_ind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121396 entries, 0 to 2121395
Data columns (total 59 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   region_abr                object        
 1   region                    object        
 2   date_heure                datetime64[ns]
 3   date                      object        
 4   heure                     object        
 5   annee_mois                object        
 6   annee                     int32         
 7   mois                      int32         
 8   mois_nom                  object        
 9   jour                      int32         
 10  jour_nom                  object        
 11  jour_numero               object        
 12  semaine                   object        
 13  jour_ferie                bool          
 14  consommation              float64       
 15  thermique                 float64       
 16  nucleaire                 float64       
 17  eolien  

In [22]:
print ("Valeurs manquantes sur dataframe final : ",df_eco2_cap_ind.isnull().sum().sum())

Valeurs manquantes sur dataframe final :  0


In [23]:
# Arrondi des valeurs des capacités et indisponibilités à 0 décimale
df_eco2_cap_ind.iloc[:, 27:43] = df_eco2_cap_ind.iloc[:, 27:43].round(0)

In [24]:
df_eco2_cap_ind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121396 entries, 0 to 2121395
Data columns (total 59 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   region_abr                object        
 1   region                    object        
 2   date_heure                datetime64[ns]
 3   date                      object        
 4   heure                     object        
 5   annee_mois                object        
 6   annee                     int32         
 7   mois                      int32         
 8   mois_nom                  object        
 9   jour                      int32         
 10  jour_nom                  object        
 11  jour_numero               object        
 12  semaine                   object        
 13  jour_ferie                bool          
 14  consommation              float64       
 15  thermique                 float64       
 16  nucleaire                 float64       
 17  eolien  

In [25]:
# Export du dataFrame df_eco2_cap_ind au format CSV
df_eco2_cap_ind.to_csv('df_eco2_cap_ind_fusion.csv', sep=';', encoding='latin-1')

In [26]:
# Export des 100 premières lignes dans un nouveau fichier CSV pour aperçu
df_eco2_cap_ind_head = df_eco2_cap_ind.head(100)
df_eco2_cap_ind_head.to_csv('df_eco2_cap_ind_first_100_lines.csv', sep=';', encoding='latin-1', index=False)